# GP surrogate model

This notebook creates a surrogate model of EPANET's water quality solver using Gaussian Process (GP) regression.


In [16]:
import sys
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.colors
default_colors = plotly.colors.qualitative.Plotly
import scipy.stats as stats
import random
from pyDOE import lhs
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
from tqdm import tqdm
from bayesian_wq_calibration.epanet import build_model, sensor_model_id, epanet_simulator, set_reaction_parameters
from bayesian_wq_calibration.mcmc import decision_variables_to_dict
from bayesian_wq_calibration.constants import TIMESERIES_DIR, RESULTS_DIR
from bayesian_wq_calibration.data import bulk_temp_adjust

### Load data

Load operational data for selected sensing period.

In [2]:
data_period = 18 # 21 calibration events (as at 31 October 2024)
try:
    flow_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-flow.csv")
    pressure_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-pressure.csv")
    wq_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-wq.csv", low_memory=False)
    cl_df = wq_df[wq_df['data_type'] == 'chlorine']
except:
    print(f"Data period {data_period} does not exist.")

Surrogate model data period.

In [3]:
surrogate_days = 2

n_total = len(flow_df['datetime'].unique())
n_surrogate = surrogate_days * 24 * 4
surrogate_range = range(n_surrogate)
surrogate_datetime = flow_df['datetime'].unique()[list(surrogate_range)]
total_range = range(n_total)
total_datetime = flow_df['datetime'].unique()[list(total_range)]

Bulk decay.

In [4]:
bulk_coeff = -0.85 # day^-1 (from bottle tests)
field_temp = wq_df[wq_df['data_type'] == 'temperature']['mean'].mean()
bulk_coeff = bulk_temp_adjust(bulk_coeff, field_temp)

Wall decay grouping.

In [5]:
grouping = 'material-velocity' # 'single', 'material', 'material-diameter', 'material-velocity'

if grouping == 'single':
    param_bounds = [(-1.0, 0.0)] # single wall decay coefficient
elif grouping == 'material':
    param_bounds = [(-1.0, 0.0), (-0.25, 0.0), (-0.1, 0.0)] # variable order: metallic, cement, plastic_unknown
elif grouping == 'material-diameter':
    param_bounds = [(-1.0, 0.0), (-0.5, 0.0), (-0.25, 0.0), (-0.1, 0.0)] # variable order: metallic_less_than_150, metallic_greater_than_150, cement, plastic_unknown
elif grouping == 'material-velocity':
    param_bounds = [(-1.0, 0.0), (-0.5, 0.0), (-0.5, 0.0), (-0.25, 0.0), (-0.25, 0.0), (-0.1, 0.0)] # variable order: metallic_low_velocity, metallic_high_velocity, cement_low_velocity, cement_high_velocity, plastic_low_velocity, plastic_high_velocity

wall_params = [random.uniform(lower, upper) for lower, upper in param_bounds]
n_params = len(wall_params)

### Surrogate model

**EPANET simulator**

Build water model using `wntr`.

In [6]:
demand_resolution = 'wwmd'
wn = build_model(flow_df[flow_df['datetime'].isin(surrogate_datetime)], pressure_df[pressure_df['datetime'].isin(surrogate_datetime)], cl_df[cl_df['datetime'].isin(surrogate_datetime)], sim_type='chlorine', demand_resolution=demand_resolution, bulk_coeff=bulk_coeff)

Get mean velocities (for 'material-velocity' grouping).

In [7]:
if grouping == 'material-velocity':
    sim_results = epanet_simulator(wn, 'velocity', cl_df[cl_df['datetime'].isin(surrogate_datetime)])
    vel_sim = sim_results.velocity.T
    mean_vel = vel_sim.mean(axis=1)
    mean_vel = mean_vel.reset_index().rename(columns={'name': 'link_id', 0: 'mean_vel'})
else:
    mean_vel = None

Define simualtor function.

In [8]:
def simulator(cl_df, wall_params, wn, grouping, mean_vel):
    wall_params = decision_variables_to_dict(grouping, wall_params)
    _wn = set_reaction_parameters(wn, grouping, wall_params, None, mean_vel)
    
    sim_type = 'chlorine'
    sim_results = epanet_simulator(_wn, sim_type, cl_df)
    cl_sim = sim_results.chlorine
    
    sensor_data = sensor_model_id('wq')
    cl_sim = cl_sim[sensor_data['model_id'].unique()]
    name_mapping = sensor_data.set_index('model_id')['bwfl_id'].to_dict()
    cl_sim = cl_sim.rename(columns=name_mapping)

    cl_sim = cl_sim.T
    cl_sim.columns = [f't_{idx+1}' for idx in range(cl_sim.shape[1])]

    cl_sim = cl_sim.drop(index=['BW1', 'BW4'], errors='ignore') # remove inlet sensors
    
    return cl_sim

In [9]:
cl_simulator = simulator(cl_df[cl_df['datetime'].isin(surrogate_datetime)], wall_params, wn, grouping, mean_vel)

**Design of experiments**

Latin Hypercube Sampling (LHS)

In [10]:
def experiments_lhs(param_bounds, n_params, n_samples):
    samples = lhs(n_params, samples=n_samples)
    scaled_samples = np.array([
        low + (high - low) * sample
        for (low, high), sample in zip(param_bounds, samples.T)
    ]).T
    return scaled_samples

In [56]:
n_samples = [10, 25, 50, 100, 200]
n_samples_idx = 2

X = experiments_lhs(param_bounds, n_params, n_samples[n_samples_idx])
Y = np.array([
    simulator(cl_df[cl_df['datetime'].isin(surrogate_datetime)], params, wn, grouping, mean_vel)
    for params in X
])
Y_flat = Y.reshape(n_samples[n_samples_idx], Y.shape[1] * Y.shape[2]) # reshape for GP model

**Gausian process model training**

Training procedure using five-fold cross-validation and `GPyTorch` modules. The following kernel's can be used:
- Radial basis function (RBF)
- Matern
- Rational quadratic

In [57]:
import torch
import gpytorch
from gpytorch.kernels import RBFKernel, MaternKernel, ScaleKernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import MultitaskGaussianLikelihood
from gpytorch.models import ExactGP

class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_tasks, kernel_type='RBF', nu=1.5):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=num_tasks
        )
        
        if kernel_type == "RBF":
            self.covar_module = gpytorch.kernels.MultitaskKernel(
                gpytorch.kernels.RBFKernel(lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(1e-1, 1e10)), num_tasks=num_tasks, rank=1
            )
        elif kernel_type == "Matern":
            self.covar_module = gpytorch.kernels.MultitaskKernel(
                gpytorch.kernels.MaternKernel(nu=nu, lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(1e-1, 1e10)), num_tasks=num_tasks, rank=1
            )
        else:
            raise ValueError(f"Unknown kernel type: {kernel_type}")

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)

Cross-validation step.

In [ ]:
# setup parameters
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)
kernel = 'RBF' # 'RBF', 'Matern'
num_iter = 50
matern_nu = 1.5

# cross-validation loop
hyperparameter_performance = []
for fold, (train_idx, validate_idx) in enumerate(kf.split(X)):

    # split the data into train and validate sets
    X_train, X_validate = X[train_idx], X[validate_idx]
    Y_train, Y_validate = Y_flat[train_idx], Y_flat[validate_idx]
    Y_train = Y_train.reshape(X_train.shape[0], -1)
    Y_validate = Y_validate.reshape(X_validate.shape[0], -1)

    # convert to torch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
    X_validate_tensor = torch.tensor(X_validate, dtype=torch.float32)
    Y_validate_tensor = torch.tensor(Y_validate, dtype=torch.float32)

    # initialize the likelihood and model
    num_tasks=Y_train.shape[1]
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=Y_train.shape[1])
    model = MultitaskGPModel(X_train_tensor, Y_train_tensor, likelihood, num_tasks, kernel_type=kernel, nu=matern_nu)

    # training loop
    model.train()
    likelihood.train()
    optimizer = torch.optim.Adam([
        {'params': model.parameters()}
    ], lr=0.1)

    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    
    for i in tqdm(range(num_iter), desc=f"Fold {fold + 1} training", unit="iter", file=sys.stdout):
        optimizer.zero_grad()
        output = model(X_train_tensor)
        loss = -mll(output, Y_train_tensor)
        loss.backward()
        optimizer.step()

    # validation step
    model.eval()
    likelihood.eval()

    print('Evaluating validation data...')

    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        predictions = likelihood(model(X_validate_tensor))
        predicted_mean = predictions.mean
        lower, upper = predictions.confidence_region()

    # compute evaluation metrics
    predicted_mean = predicted_mean.numpy()
    rmse = np.sqrt(mean_squared_error(Y_validate, predicted_mean))
    mae = mean_absolute_error(Y_validate, predicted_mean)
    maxae = np.max(np.abs(Y_validate - predicted_mean))
    hyperparameter_performance.append({
        "fold": fold + 1,
        "rmse": rmse,
        "mae": mae,
        "maxae": maxae,
        "length_scale": model.covar_module.data_covar_module.lengthscale.item(),
        "variance": model.covar_module.task_covar_module.var.detach().numpy()
    })
    
    print(f"Fold {fold + 1} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, MaxAE: {maxae:.4f}")

Fold 1 training: 100%|████████████████████████████████████████████████████████████████| 50/50 [01:43<00:00,  2.08s/iter]
Evaluating validation data...


Output cross-validation results.

In [ ]:
print("\nCross-validation results:")
rmse_mean, mae_mean, maxae_mean = np.mean([hp["rmse"] for hp in hyperparameter_performance]), \
                                   np.mean([hp["mae"] for hp in hyperparameter_performance]), \
                                   np.mean([hp["maxae"] for hp in hyperparameter_performance])
rmse_std, mae_std, maxae_std = np.std([hp["rmse"] for hp in hyperparameter_performance]), \
                               np.std([hp["mae"] for hp in hyperparameter_performance]), \
                               np.std([hp["maxae"] for hp in hyperparameter_performance])

print(f"Mean RMSE: {rmse_mean:.4f} ± {rmse_std:.4f}")
print(f"Mean MAE: {mae_mean:.4f} ± {mae_std:.4f}")
print(f"Mean MaxAE: {maxae_mean:.4f} ± {maxae_std:.4f}")

for hp in hyperparameter_performance:
    print(f"Fold {hp['fold']} - Length Scale: {hp['length_scale']}, Variance: {hp['variance']}")

# select best hyperparameters
best_fold = min(hyperparameter_performance, key=lambda x: x['rmse'])
best_length_scale = best_fold['length_scale']
best_variance = best_fold['variance']
print(f"Best fold based on RMSE: {best_fold['fold']}")
print(f"Best length scale: {best_length_scale}")
print(f"Best variance: {best_variance}")

Re-train model with optimized hyperparameters.

In [ ]:
# insert code here...

Model testing.

In [ ]:
def random_samples(param_bounds, n_samples):
    n_params = len(param_bounds)
    samples = np.random.uniform(
        low=[low for low, high in param_bounds],
        high=[high for low, high in param_bounds],
        size=(n_samples, n_params)
    )
    return samples

In [ ]:
# create testing dataset
n_tests = 20
X_test = random_samples(param_bounds, n_tests)
Y_test = np.array([
    simulator(cl_df[cl_df['datetime'].isin(surrogate_datetime)], params, wn, grouping, mean_vel)
    for params in X_test
])
Y_test_flat = Y_test.reshape(n_tests, Y_test.shape[1] * Y_test.shape[2])

# model evaluation
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test_flat, dtype=torch.float32)

model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions = likelihood(model(X_test_tensor))
    predicted_mean = predictions.mean
    lower, upper = predictions.confidence_region()

Y_pred_flat_mean = predicted_mean.numpy()
Y_pred_mean = Y_pred_flat_mean.reshape(n_tests, Y_test.shape[1], Y_test.shape[2])
Y_pred_flat_lower = lower.numpy()
Y_pred_lower = Y_pred_flat_lower.reshape(n_tests, Y_test.shape[1], Y_test.shape[2])
Y_pred_flat_upper = upper.numpy()
Y_pred_upper = Y_pred_flat_upper.reshape(n_tests, Y_test.shape[1], Y_test.shape[2])

Testing performance metrics

In [ ]:
rmse = np.sqrt(np.mean((Y_test_flat - Y_pred_flat) ** 2))
print(f"Root mean squared error: {mse}")
mae = np.mean(np.abs(Y_test_flat - Y_pred_flat))
print(f"Mean absolute error: {mae}")
maxae = np.max(np.abs(Y_test_flat - Y_pred_flat))
print(f"Maximum absolute error: {maxae}")

# parity plot of surrogate v. simulator
fig = go.Figure(data=go.Scatter(
    x=Y_test_flat.flatten(),
    y=Y_pred_flat_mean.flatten(),
    mode='markers',
    marker=dict(size=6, opacity=0.6),
))
fig.update_layout(
    xaxis_title="Simulator [mg/L]",
    yaxis_title="Surrogate [mg/L]",
    template="simple_white",
    width=600,
    height=500
)
fig.show()

# histogram plot of errors
errors = (Y_test_flat - Y_pred_flat_mean).flatten()
fig = go.Figure(data=[go.Histogram(x=errors, nbinsx=40)])
fig.update_layout(
    xaxis_title="Error [mg/L]",
    yaxis_title=f"Frequency (n={len(errors)})",
    template="simple_white",
    width=600,
    height=400
)
fig.show()

# cdf plot of absolute errors
absolute_errors = np.abs(Y_test_flat - Y_pred_flat_mean).flatten()
sorted_errors = np.sort(absolute_errors)
cdf_values = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=sorted_errors,
    y=cdf_values,
    mode='lines',
))
fig.update_layout(
    xaxis_title="Absolute Error [mg/L]",
    yaxis_title="Cumulative distribution",
    template="simple_white",
    width=600,
    height=400
)
fig.show()

In [ ]:
selected_sensor_idx = 3
sensor_name = cl_simulator.index[selected_sensor_idx]

fig = go.Figure()

actual_data = Y_test[:, selected_sensor_idx, :]
predicted_data = Y_pred[:, selected_sensor_idx, :]

for exp_idx in range(n_samples-n_train):
    color = default_colors[exp_idx % len(default_colors)]
    fig.add_trace(
        go.Scatter(
            x=surrogate_datetime,
            y=actual_data[exp_idx, :],
            mode='lines',
            name=f"Simulator (Exp {exp_idx + 1})",
            line=dict(color=color, dash='solid'),
            showlegend=True
        )
    )
    fig.add_trace(
        go.Scatter(
            x=surrogate_datetime,
            y=predicted_data[exp_idx, :],
            mode='lines',
            name=f"GP Model (Exp {exp_idx + 1})",
            line=dict(color=color, dash='dash'),
            showlegend=True
        )
    )
fig.update_yaxes(
    title_text="Chlorine [mg/L]",
    rangemode="tozero"
)
fig.update_layout(
    height=600,  # Fixed height since there's only one plot
    template='simple_white',
    legend_title_text='',
    title=f"GP model validation sensor {sensor_name}"
)
fig.show()

### Define loglikelihood score function

In [ ]:
# def loglikelihood(sim, cl_df):
    
#     sensor_data = sensor_model_id('wq')
#     bwfl_ids = [sensor for sensor in sensor_data['bwfl_id'].unique() if sensor not in ['BW1', 'BW4']]
#     datetime = cl_df['datetime'].unique()[96:]
    
#     loglikelihood = 0
#     for name in bwfl_ids:   
#         _sim = sim[name].values
#         data = cl_df.loc[cl_df['bwfl_id'] == name, 'mean'].values
#         mask = ~np.isnan(sim) & ~np.isnan(data) & (np.arange(len(sim)) >= 96)
#         mse += (1 / (len(datetime) * len(bwfl_ids))) * np.sum((sim[mask] - data[mask]) ** 2)
    
#     return None